In [ ]:
%pip install -q social_agents/

In [ ]:
%pip install -qU langchain_mistralai

In [ ]:
%pip show social_agents

In [ ]:
%pip install -U pydantic
%pip install -U "langchain[together]"
%pip install -U "langchain"
%pip install -U "langgraph"
%pip install -U "langgraph-mistralai"

In [ ]:
import tqdm
# Reading the data
import pandas as pd

from social_agents.utils import get_st_data


for key, line in tqdm.tqdm(get_st_data("sample").items()):
    print(key)

    print(line['intervention'])
    input_text = line['intervention']
    break


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

## Prerequisites
env, LLM, ...

# Shared Task: Critical thinking generation
## Loadind Data


In [ ]:
import os, getpass
from langchain.chat_models import init_chat_model

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("MISTRAL_API_KEY")
_set_env("TOGETHER_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "shared_task_critical_questions"

#init_chat_model("meta-llama/Llama-3.2-3B-Instruct-Turbo", model_provider="together", temperature= 0.7)
os.environ["MISTRAL_API_KEY"]

##  1. Zero Shot LLM to start with o3-mini

In [ ]:

from social_agents.agent_builder import BasicCQModel
model_name = "o3-mini"

basic_agent = BasicCQModel(llm_name = model_name, temperature=None ) #interrupt_before=['human_feedback'], checkpointer=memory)


display(Image(basic_agent.graph.get_graph(xray=1).draw_mermaid_png()))

# RUN
# basic_agent.run_experiment()



### Evaulate 

- Run shell 

```shell

python3 eval_scripts/evaluation.py \
    --metric similarity \
    --input_path data_splits/validation.json \
    --submission_path output/elbaff_experiment/output_gpt-4o-mini-2024-07-18social_n2_Te_Sr.json \
    --threshold 0.6 

```

- OUTPUT : output/output_o3-mini_temperatureNA._eval_similarity_06.json



**Overall count**

| **Questions Labels** |  **#**  | **ratio** |
|:--------------------|-------:|---------:|
| useful               | **329** |  **0,59** |
| unhelpful            |      63 |      0,11 |
| Invalid              |       8 |      0,01 |
| Not able to evaluate |     158 |      0,28 |
| **Total**            | **558** |         1 |


**Overall count within each argument**

| **n/3 useful questions per arg** | **# of arguments** | **ratio** |
|:-------------------------------:|-------------------:|:---------:|
|                               0/3 |                 17 |      0,10 |
|                               1/3 |                 51 |      0,27 |
|                               2/3 |             **76** |      0,40 |
|                               3/3 |                 42 |      0,23 |
| **Total**                       |            **186** |         1 |

## Social Agents


In [ ]:
%pip show langchain-core

In [ ]:
import pandas as pd
import os

from social_agents import helper
from social_agents.agent_builder import  CQSTAbstractAgent, SocialAgentBuilder

EXPERIMENT_SETTINGS_FILE_PATH = "output/elbaff_experiment/experiment_settings.csv"
override = False
if not os.path.exists(EXPERIMENT_SETTINGS_FILE_PATH) or override:
    print("generating exp settings file")
    exps_df = helper.generate_experiment_settings()
    exps_df.to_csv(EXPERIMENT_SETTINGS_FILE_PATH, index=False)

experiment_settings = pd.read_csv(EXPERIMENT_SETTINGS_FILE_PATH)
experiment_settings.head()

# exp_row = experiment_settings.iloc[20].to_dict()
# exp_row

In [ ]:
experiment_settings = experiment_settings.sort_values(by=["rounds", "number_of_agents"])

In [ ]:
from langchain.chat_models import init_chat_model

from social_agents.data_model import CriticalQuestionList
prompt = """
# Context:
You are tasked with generating critical questions that examine the underlying assumptions and logic of an argumentative text. Given a real debate argument, your goal is to assess its acceptability and potential fallacies.

# Argument:

'''
CLINTON: "nine million people lost their jobs
Five million people lost their homes
$ 13 trillion in family wealth was wiped out
Now , we have come back from that abyss
it has not been easy
we 're now on the precipice of having a potentially much better economy
the last thing we need to do is to go back to the policies that failed us in the first place
Independent experts have looked at what I 've proposed
looked at what Donald 's proposed
I intend to get it done
Take clean energy
Some country is going to be the clean- energy superpower of the 21st century
Donald thinks that climate change is a hoax perpetrated by the Chinese
I think it 's real"
'''

# Instructions:
- Read the argument carefully.
- Generate three critical questions that:
    - Uncover the assumptions behind the argument's premises.
    - Challenge the strength of its inferences.
- Ensure your questions are specific to the given argument and not generic.

"""
rsponse = init_chat_model("mistral-small-2503",
                model_provider="mistralai",
                temperature=0.7).with_structured_output(CriticalQuestionList, include_raw=True).invoke(prompt)
rsponse
# cq_ = CriticalQuestionList(rsponse)
#print(type(cq_))


In [ ]:
import ast
import time
from social_agents import data_model
import ast
from tqdm import tqdm


from IPython.display import Image, display

#llm_name = "gpt-4o-mini-2024-07-18"# "o3-mini-2025-01-31"
#llm_name_short = "gpt-4o-mini_"
llm_dic = {
    "openai_o3mini": "o3-mini-2025-01-31",
    "gpt-4o-mini_": "gpt-4o-mini-2024-07-18",
    #"llam3b_": "meta-llama/Llama-3.2-3B-Instruct-Turbo",
    "llama8b_": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "mistral24b_": "mistral-small-2503",#"mistralai/Mixtral-8x7B-Instruct-v0.1",
    #"llama8blite_": "meta-llama/Meta-Llama-3-8B-Instruct-Lite",
    "llama70b_": "meta-llama/Llama-3.3-70B-Instruct-Turbo"

}

#mistralai/Mixtral-8x22B-Instruct-v0.1
#mistralai/Mistral-7B-Instruct-v0.3
#mistralai/Mixtral-8x7B-Instruct-v0.1
temperature = 0.7
llm_name_short = "mistral24b_"
llm_name = llm_dic[llm_name_short]
def run_all_exp_settings(experiment_settings):
    all_done = False
    for _, row in tqdm(experiment_settings.iterrows(), total=len(experiment_settings)):
        exp_name = row["experiment_name"].format(llm_name= llm_name_short)
        if os.path.exists(f"{SocialAgentBuilder.ROOT_FOLDER}output_{exp_name}.json"):
            print("Experiment ", exp_name, "already done!")
            continue
        if True:##"easy_going" in list(ast.literal_eval(row["traits"])): #and len(list(ast.literal_eval(row["traits"]))) <3 and len(list(ast.literal_eval(row["strategies"]))) < 3:
            print("="*20)
            print(row.to_dict())
            print("EXPERIMENT NAME: ", exp_name)
            print("="*20)

            social_agent = SocialAgentBuilder(
                model_thread_id=row["thread_id"],
                llm_name = llm_name,
                llm_num = row["number_of_agents"],
                experiment_name= exp_name,
                temperature=temperature,
                collaborative_strategy=list(ast.literal_eval(row["strategies"])),
                agent_trait_lst=list(ast.literal_eval(row["traits"])))
            #display(Image(social_agent.graph.get_graph(xray=1).draw_mermaid_png()))

            social_agent.run_experiment(data_type = "validation", save= True)
            print(f'finished {exp_name}')
    all_done = True
    return all_done
delay_in_sec = 1
all_done = False
while True and not all_done: 
    try:###
        all_done = run_all_exp_settings(experiment_settings=experiment_settings)
    except Exception as e:
        all_done = False
        print(f"Exception caught: {e}, retrying in {delay_in_sec} seconds...")
        time.sleep(delay_in_sec)


# Best w perfomer
-> "mistral 8x7b"
-> "4o"
small are 

# Future prompting
few shot 

# Best for validators
llama 3.1 3b --> w aggregator



In [ ]:
# small model
